# Core Features: Lines and Planes in Parameter Space

This example covers the basic features of the `loss-landscapes` library, i.e. evaluating a model's loss function along lines or planes in parameter space in order to produce visualizations of the loss landscape.

In [ ]:
# libraries
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import numpy as np
import torch
import torch.nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as datasets
from tqdm import tqdm

matplotlib.rcParams['figure.figsize'] = [18, 12]

# code from this library - import the lines module
import loss_landscapes
import loss_landscapes.utils
import loss_landscapes.evaluators.torch as evaluators

## 1. Preliminary: Classifying MNIST

This notebook demonstrates how to accomplish a simple task: visualizing the loss landscape of a small fully connected feed-forward neural network on the MNIST image classification task. In this section the preliminaries (the model and the training procedure) are setup.

In [ ]:
# training hyperparameters
IN_DIM = 28 * 28
OUT_DIM = 10
LR = 10 ** -2
BATCH_SIZE = 512
EPOCHS = 25
# contour plot resolution
STEPS = 40

The cells in this section contain no code specific to the `loss-landscapes` library.

In [ ]:
class MLPSmall(torch.nn.Module):
    """ Fully connected feed-forward neural network with one hidden layer. """
    def __init__(self, x_dim, y_dim):
        super().__init__()
        self.linear_1 = torch.nn.Linear(x_dim, 32)
        self.linear_2 = torch.nn.Linear(32, y_dim)

    def forward(self, x):
        h = F.relu(self.linear_1(x))
        return F.softmax(self.linear_2(h), dim=1)


class Flatten(object):
    """ Transforms a PIL image to a flat numpy array. """
    def __call__(self, sample):
        return np.array(sample, dtype=np.float32).flatten()    
    

def train(model, optimizer, criterion, train_loader, epochs):
    """ Trains the given model with the given optimizer, loss function, etc. """
    model.train()
    # train model
    for _ in tqdm(range(epochs), 'Training'):
        for count, batch in enumerate(train_loader, 0):
            optimizer.zero_grad()
            x, y = batch

            pred = model(x)
            loss = criterion(pred, y)
            loss.backward()
            optimizer.step()

    model.eval()

The model is then trained, producing a start point and an end point for plotting purposes.

In [ ]:
# download MNIST and setup data loaders
mnist_train = datasets.MNIST(root='../data', train=True, download=True, transform=Flatten())
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=BATCH_SIZE, shuffle=False)

# define model and deepcopy initial model
model = MLPSmall(IN_DIM, OUT_DIM)
optimizer = optim.Adam(model.parameters(), lr=LR)
criterion = torch.nn.CrossEntropyLoss()

## 2. Points in Parameter Space
The state of a neural network model represents a point in parameter space. Several functions in the `loss-landscapes` library require the user to provide instances of neural network models as arguments, where the model is taken to represent the point in parameter space corresponding to its parameters' state. Representing points as simply the models they relate to reduces the complexity of the client code.

In the cell below, a copy of the model before training is made, to preserve the model's initial state.

In [ ]:
# stores the initial point in parameter space
model_initial = loss_landscapes.utils.deepcopy_model(model, 'torch')

Two common points of interest are the model initialization, and the model's final parameters after training. Similarly to the cell above, we can make a copy of the model after training. A copy of the model can be made at any time - this is up to the user, of course.

In [ ]:
train(model, optimizer, criterion, train_loader, EPOCHS)

model_final = loss_landscapes.utils.deepcopy_model(model, 'torch')

## 3. Linear Interpolations of Loss between Two Points

In the previous section, copies of the model before and after training were obtained. A common use case is to evaluate the model loss at a number of equidistant points along the straight line connecting the two points. Although care must be taken in interpreting such plots, the idea is to gain an insight into the smoothness of the landscape.

The user might also be interested in collecting any other quantity along this line. The `loss-landscapes` library abstracts these details with entities called `Evaluator`s, which compute some quantity about a model for its current parameters. A number of pre-defined evaluators is provided in the `loss_landscapes.evaluators` package, and the user is free to write custom evaluators. See the relevant documentation.

An important evaluator for use with PyTorch models is the torch `LossEvaluator`, which applies a PyTorch loss function to a PyTorch model and returns the value produced. 

In [ ]:
# data that the evaluator will use when evaluating loss
x, y = iter(train_loader).__next__()
loss_evaluator = evaluators.LossEvaluator(criterion, x, y)

# compute loss data
loss_data = loss_landscapes.linear_interpolation(model_initial, model_final, loss_evaluator, STEPS)

With the computed loss data, a linear interpolation plot can be rendered.

In [ ]:
plt.plot([1/STEPS * i for i in range(STEPS)], loss_data)
plt.title('Linear Interpolation of Loss')
plt.xlabel('Interpolation Coefficient')
plt.ylabel('Loss')
axes = plt.gca()
# axes.set_ylim([2.300,2.325])
plt.show()

## 4. Planar Approximations of Loss Around a Point

Another core use for the library is producing 2-dimensional approximations of the loss-landscape topology around a point in parameter space. This is accomplished by sampling two random direction vectors in parameter space, and computing the loss at a number of points on the plane defined by the two vectors:

In [ ]:
loss_data_fin = loss_landscapes.random_plane(model_final, loss_evaluator, 10, STEPS, 'layer')

The loss values on this plane can be visualized in an intuitive and interpretable manner using contour plots or 3D surface plots:

In [ ]:
plt.contour(loss_data_fin, levels=50)
plt.title('Loss Contours around Trained Model')
plt.show()

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')
X = np.array([[j for j in range(STEPS)] for i in range(STEPS)])
Y = np.array([[i for _ in range(STEPS)] for i in range(STEPS)])
ax.plot_surface(X, Y, loss_data_fin, rstride=1, cstride=1, cmap='viridis', edgecolor='none')
ax.set_title('Surface Plot of Loss Landscape')
fig.show()